**Generative AI**

* Generating a new sets of data using existing data

**Generative uses two types of Models**

* GAN => Generative Adversal Network => Two parsts => 1. Generator
    * 1. Generator - Generates or creats new data samples such as images or text
    * 2. Discriminator - Tries to Distinguish between real data and the data generated by generator
* Autoregressive Models => used for generating sequence of data such as text or music.

In [5]:
import openai
from secret_key import openai_key
from langchain.llms import OpenAI

import os
os.environ['OPENAI_API_KEY'] = openai_key

In [6]:
llm = OpenAI(temperature = 0.7)
name = llm("write a 4 line poem on India")
print(name)



India, a nation of colors,
Vibrant culture and traditions true,
A people that celebrate life,
A country that is ever so new.


In [7]:
from langchain.prompts import PromptTemplate

promt_template_name = PromptTemplate(
        input_variables = ['city'],
        template = "I want some tourist place in {city}. Suggest Something"
    )

promt_template_name.format(city = "Jaipur")

'I want some tourist place in Jaipur. Suggest Something'

In [8]:
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = promt_template_name)
result = chain.run("Delhi")
print(result)



1. Red Fort
2. Qutub Minar
3. India Gate
4. Lotus Temple
5. Humayun's Tomb
6. Jama Masjid
7. Akshardham Temple
8. Rashtrapati Bhavan
9. Jantar Mantar
10. Purana Qila


**Simple Sequential Chain**

In [9]:
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature = 0.7)

promt_template_area = PromptTemplate(
        input_variables = ['area'],
        template = "i want one city name for tourism in {area}"
    )

area_chain = LLMChain(llm = llm, prompt = promt_template_area)

promt_template_city = PromptTemplate(
        input_variables = ['city'],
        template = "suggest some tourist place in {city}"
    )

city_chain = LLMChain(llm = llm, prompt = promt_template_city)

In [10]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [area_chain,city_chain])
response = chain.run("Northern India")
print(response)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xoGm0GNO0nfNytJQp6R6icUj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


: Taj Mahal, Agra Fort, Fatehpur Sikri

Jaipur: Hawa Mahal, Amber Fort, City Palace

Goa: Calangute Beach, Baga Beach, Anjuna Beach

Delhi: Red Fort, India Gate, Humayun's Tomb

Mumbai: Gateway of India, Marine Drive, Haji Ali Dargah


In [11]:
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature = 0.7)

promt_template_area = PromptTemplate(
        input_variables = ['area'],
        template = "i want one city name for tourism in {area}"
    )

area_chain = LLMChain(llm = llm, prompt = promt_template_area , output_key="city")

promt_template_city = PromptTemplate(
        input_variables = ['city'],
        template = "suggest some tourist place in {city}"
    )

city_chain = LLMChain(llm = llm, prompt = promt_template_city , output_key = "places")

In [12]:
from langchain.chains import SequentialChain
chain = SequentialChain(chains = [area_chain,city_chain],
                       input_variables = ['area'],
                       output_variables = ['city' , 'places'])
response = chain({"area" : "Northern India"})

for result in response.values():
    print(result)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-xoGm0GNO0nfNytJQp6R6icUj on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


Northern India


Agra
 - Taj Mahal
Jaipur - Amber Fort, Hawa Mahal
Varanasi - Kashi Vishwanath Temple, Dashashwamegh Ghat
Udaipur - City Palace, Lake Pichola
Goa - Baga Beach, Calangute Beach
Kerala - Munnar, Alleppey Backwaters
Andaman and Nicobar Islands - Radhanagar Beach, Cellular Jail


In [29]:
print(response)

{'area': 'Southern India', 'city': '\n\nMysore.', 'places': "\n\n1. Mysore Palace\n2. Brindavan Gardens\n3. Mysore Zoo\n4. Chamundi Hill\n5. Jaganmohan Palace\n6. Karanji Lake\n7. St. Philomena's Church\n8. Rail Museum\n9. GRS Fantasy Park\n10. Srirangapatna"}


In [31]:
print(type(response))
print(response.keys())
print(response.values())

<class 'dict'>
dict_keys(['area', 'city', 'places'])
dict_values(['Southern India', '\n\nMysore.', "\n\n1. Mysore Palace\n2. Brindavan Gardens\n3. Mysore Zoo\n4. Chamundi Hill\n5. Jaganmohan Palace\n6. Karanji Lake\n7. St. Philomena's Church\n8. Rail Museum\n9. GRS Fantasy Park\n10. Srirangapatna"])


In [33]:
for result in response.values():
    print(result)

Southern India


Mysore.


1. Mysore Palace
2. Brindavan Gardens
3. Mysore Zoo
4. Chamundi Hill
5. Jaganmohan Palace
6. Karanji Lake
7. St. Philomena's Church
8. Rail Museum
9. GRS Fantasy Park
10. Srirangapatna


**Agent**

In [39]:
!pip3 install wikipedia

Using legacy 'setup.py install' for wikipedia, since package 'wheel' is not installed.
    Running setup.py install for wikipedia ... done
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [45]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(["wikipedia"] , llm = llm)

agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION)

agent.run("Who won Last World Test Championship final")

'New Zealand won the Last World Test Championship final.'